## LIBRERIAS

In [91]:
import pandas as pd
import pickle
from scipy.stats import poisson
from pprint import pprint
from IPython.display import display
import numpy as np

## FUNCIONES

In [115]:
def predict_points(home,away):
    # Reviso los 2 equipos en el Index
    
    if home in df_TeamStrength.index and away in df_TeamStrength.index:
        # goals_scored * goals_conceded
        # Lambda Goles esperados Local
        lamb_home= df_TeamStrength.at[home,"GoalsScored"]*df_TeamStrength.at[away,"GoalsConceded"]
        # Lambda Goles Esperados Visitante
        lamb_away= df_TeamStrength.at[away,"GoalsScored"]*df_TeamStrength.at[home,"GoalsConceded"]
        
        # Inicializo probabilidades ganar, perder, empate
        prob_home,prob_away,prob_draw=0,0,0

        # Hago una simulacion de las combinaciones de goles posibles comb 0 a 10 en x e y
        for x in range (0,11): #number of goals home team
            for y in range(0,11): #number of goals away team
                p=poisson.pmf(x,lamb_home)*poisson.pmf(y,lamb_away)
                if x==y:
                    prob_draw+=p
                elif x>y:
                    prob_home+=p
                else: 
                    prob_away+=p
        
        point_home=3*prob_home+prob_draw
        point_away=3*prob_away+prob_draw
        return(float(point_home),float(point_away))
    else:
        return(0,0)

def simulate_group_stage(dict_table, df_fixture_groups):

    # Inicializo los puntos en 0, para que cada vez que los corran, no se acumulen
    # Recordar que tables es un diccionario de dfs
    for table in dict_table.values(): # Table.values me devuelve el valor que un df
        table['Pts'] = 0.0
        table['Pts'] = table['Pts'].astype(float) # lo transformo en floats antes de hacer la sumatoria, para evitar errores de tipo

    for group, table in dict_table.items():

        #Me guardo los equipos
        teams = table["Selección"].values

        # Busco los grupos que en el fixture, si esta en los fiturs
        df_group = df_fixture_groups[
            (df_fixture_groups['home'].isin(teams)) &
            (df_fixture_groups['away'].isin(teams))
        ]

        #Aca itero los grupos
        for row in df_group.itertuples(): # aca traigo cada row del df del df de fixture
            home = row.home
            away = row.away

            # Predigo los puntos llamo a la funcion
            points_home, points_away = predict_points(home, away)

            # sumo los puntos a cada uno
            table.loc[table["Selección"] == home, 'Pts'] += points_home
            table.loc[table["Selección"] == away, 'Pts'] += points_away

        
        dict_table[group] = (
            table
            .sort_values('Pts', ascending=False)
            .reset_index(drop=True)
        )
   
        dict_table[group]= dict_table[group][["Selección","Pts"]]
        dict_table[group]=dict_table[group].round(0)
    return dict_table

def resolver_mejor_tercero(valor, df_third):
    
    if not isinstance(valor, str):
        return valor
    
    if "3" not in valor:
        return valor
    
    if "/" not in valor:
        return reemplazos.get(valor, valor)
    
    # Caso tipo 3C/D/E
    grupos = valor.replace("3", "").split("/")
    
    candidatos = df_third[df_third["Group"].isin(grupos)]
    
    mejor = candidatos.sort_values(
        by=["Pts"],
        ascending=False
    ).iloc[0]
    
    return mejor["Selección"]

def simular_eliminatoria(df_fixture):
    
    df_fixture = df_fixture.copy()  # evitamos modificar el original
    
    for nro_partido, partido in df_fixture.iterrows():
        
        equipo_home = partido["home"]
        equipo_away = partido["away"]

        goles_home, goles_away = predict_points(equipo_home, equipo_away)

        goles_home = float(goles_home)
        goles_away = float(goles_away)

        df_fixture.loc[nro_partido, "score"] = f"{goles_home}-{goles_away}"
        df_fixture.loc[nro_partido, "Goals_home"] = goles_home
        df_fixture.loc[nro_partido, "Goals_away"] = goles_away
        if goles_home>goles_away:
            df_fixture.loc[nro_partido, "Winner"] = partido["home"]
        else:
            df_fixture.loc[nro_partido, "Winner"] = partido["away"]


    return df_fixture

def asignar_ganador(df_ronda_ant, df_ronda):
    
    df_ronda = df_ronda.copy()
    
    # Crear diccionario: "Ganador Partido X" → Equipo
    reemplazos = {
        f"Ganador Partido {row['match_number']}": row["Winner"]
        for _, row in df_ronda_ant.iterrows()
    }
    
    # Reemplazar en columnas home y away
    df_ronda["home"] = df_ronda["home"].replace(reemplazos)
    df_ronda["away"] = df_ronda["away"].replace(reemplazos)
    
    return df_ronda

def asignar_perdedor(df_ronda_ant, df_ronda):
    
    df_ronda = df_ronda.copy()
    
    reemplazos = {
        f"Perdedor Partido {row['match_number']}":
            row["away"] if row["Winner"] == row["home"] else row["home"]
        for _, row in df_ronda_ant.iterrows()
    }
    
    df_ronda["home"] = df_ronda["home"].replace(reemplazos)
    df_ronda["away"] = df_ronda["away"].replace(reemplazos)
    
    return df_ronda




## IMPORTO DATAFRAMES Y DICCIONARIOS

In [93]:
dict_table=pickle.load(open("Data\FIFA_World_Cup_Groups.pkl","rb"))
df_historical_data=pd.read_csv("Data\clean_fifa_worldcup_matches.csv")
df_fixture=pd.read_csv("Data\Clean_fifa_worldcup_fixture.csv")

<>:1: SyntaxWarning: "\F" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\F"? A raw string is also an option.
<>:2: SyntaxWarning: "\c" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\c"? A raw string is also an option.
<>:3: SyntaxWarning: "\C" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\C"? A raw string is also an option.
<>:1: SyntaxWarning: "\F" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\F"? A raw string is also an option.
<>:2: SyntaxWarning: "\c" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\c"? A raw string is also an option.
<>:3: SyntaxWarning: "\C" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\C"? A raw string is also an option.
C:\Users\Usuario\AppData\Local\Temp\ipykernel_2728\1385904799.py:1: SyntaxWarning:

## Calculo fortaleza de cada equipo

In [94]:
# dividir df en df_home and df_away
df_home=df_historical_data[["HomeTeam","HomeGoals","AwayGoals"]]
df_away=df_historical_data[["AwayTeam","HomeGoals","AwayGoals"]]


In [95]:
# Renombrar columnas
df_home.rename(columns={"HomeTeam":"Team","HomeGoals":"GoalsScored","AwayGoals":"GoalsConceded"},inplace=True)
df_away.rename(columns={"AwayTeam":"Team","HomeGoals":"GoalsConceded","AwayGoals":"GoalsScored"},inplace=True)


In [96]:
# Concateno ambas tablas y calculo la media
df_TeamStrength=pd.concat([df_home,df_away],ignore_index=True).groupby("Team").mean()
df_TeamStrength.sort_values(by="Team")

,GoalsScored,GoalsConceded
Team,,
Alemania,2.020000,1.060000
Alemania Democrática,0.833333,0.833333
Alemania Federal,2.112903,1.241935
Angola,0.333333,0.666667
Arabia Saudita,0.736842,2.315789
...,...,...
Ucrania,1.000000,1.400000
Unión Soviética,1.709677,1.096774
Uruguay,1.508475,1.288136


## Justificación del uso de la Distribución de Poisson

Utilizo la distribución de Poisson para modelar la cantidad de goles en un partido de fútbol, ya que esta distribución es apropiada para describir el número de eventos que ocurren en un intervalo fijo de tiempo o espacio bajo ciertas condiciones.

En este caso:

Evento: Gol.

Intervalo: 90 minutos de partido.

Variable aleatoria: Número de goles anotados por un equipo en un partido.

📌 Supuestos de la Distribución de Poisson aplicados al fútbol
1️⃣ Los eventos son contables

El número de goles es una variable discreta y contable (0, 1, 2, 3, ...).
Por lo tanto, cumple con el requisito de que los eventos puedan cuantificarse.

2️⃣ Independencia entre eventos

La distribución de Poisson asume que la ocurrencia de un evento no afecta la probabilidad de que ocurra otro.

En el contexto del fútbol, esto puede ser debatible (un gol puede modificar el planteo táctico del partido).
Sin embargo, para simplificar el modelo, se asume independencia entre goles, lo cual es una aproximación común en modelos estadísticos deportivos.

3️⃣ Tasa de ocurrencia constante (λ constante)

La distribución de Poisson supone que la tasa promedio de ocurrencia (λ) es constante durante el intervalo de observación.

En este modelo, se asume que cada equipo tiene una tasa promedio de gol constante a lo largo de los 90 minutos.
Esta tasa puede estimarse a partir de datos históricos (promedio de goles por partido).

4️⃣ No ocurren dos eventos exactamente al mismo instante

La probabilidad de que dos goles ocurran exactamente en el mismo instante es prácticamente nula.
Dado que el tiempo es continuo, este supuesto se considera razonable.

📌 Conclusión

Bajo estos supuestos, la distribución de Poisson resulta adecuada para modelar el número de goles en un partido de fútbol, ya que:

Modela eventos discretos

En intervalos fijos

Con una tasa promedio conocida

Y baja probabilidad de ocurrencia simultánea

Este enfoque es ampliamente utilizado en modelos predictivos deportivos, especialmente en predicción de resultados y simulación de partidos.

### La formula a utilizar es 

$$
P(X = x) = \frac{\lambda^{x} e^{-\lambda}}{x!}
$$

Testeo modelo

In [97]:
print("Argentina vs México:", predict_points("Argentina", "México"))
print("Inglaterra vs Estados Unidos:", predict_points("Inglaterra", "Estados Unidos"))
print("Qatar vs Ecuador:", predict_points("Qatar", "Ecuador"))


Argentina vs México: (2.325004654272974, 0.5268219802961219)
Inglaterra vs Estados Unidos: (2.27631479427348, 0.5561408911928929)
Qatar vs Ecuador: (0, 0)


In [98]:
## LIMPIO TABLA DE GRUPOS

In [99]:
# Transformo el score en un sting , Extraigo  el numero de partido  y transofrmo match_number en int
df_fixture["match_number"] = df_fixture["score"].str.extract(r"(\d+)").astype(int)
# Reordeno por numero de partido
df_fixture = df_fixture.sort_values(by="match_number")
df_fixture

,home,score,away,year,match_number
0,México,Partido 1,Sudáfrica,2026,1
1,Corea del Sur,Partido 2,A4,2026,2
6,Canadá,Partido 3,B2,2026,3
18,Estados Unidos,Partido 4,Paraguay,2026,4
13,Haití,Partido 5,Escocia,2026,5
...,...,...,...,...,...
99,Ganador Partido 95,Partido 100,Ganador Partido 96,2026,100
100,Ganador Partido 97,Partido 101,Ganador Partido 98,2026,101
101,Ganador Partido 99,Partido 102,Ganador Partido 100,2026,102
102,Perdedor Partido 101,Partido 103,Perdedor Partido 102,2026,103


# Divido en grupos

In [100]:
df_fixture_groups=df_fixture[:72].copy()
df_fixture_dieciseisavos=df_fixture[72:88].copy()
df_fixture_octavos=df_fixture[88:96].copy()
df_fixture_cuartos=df_fixture[96:100].copy()
df_fixture_semis=df_fixture[100:102].copy()
df_fixture_final=df_fixture[102:104].copy()

# Simulacion de grupos

In [101]:
dict_table = simulate_group_stage(dict_table, df_fixture_groups)
for group, table in dict_table.items():
    pprint(group)
    display(table)

'Grupo A'


,Selección,Pts
0,Sudáfrica,3.0
1,México,3.0
2,Corea del Sur,2.0
3,Ganador Ruta D UEFA,0.0


'Grupo B'


,Selección,Pts
0,Suiza,5.0
1,Canadá,2.0
2,Catar,1.0
3,Ganador Ruta A UEFA,0.0


'Grupo C'


...

,Selección,Pts
0,Argentina,4.0
1,Austria,3.0
2,Argelia,2.0
3,Jordania,0.0


'Grupo K'


,Selección,Pts
0,Portugal,2.0
1,Colombia,1.0
2,Ganador Repesca Llave A,0.0
3,Uzbekistán,0.0


'Grupo L'


,Selección,Pts
0,Inglaterra,6.0
1,Croacia,6.0
2,Ghana,4.0
3,Panamá,0.0


## ARMO LAS LLAVES Y SIMULO DIECISEISAVOS

In [102]:
#######################################################
# 1️⃣ OBTENER TABLA GLOBAL DE TERCEROS
########################################################

third_teams = []

for group in dict_table:
    df_group = dict_table[group]
    letra = group.split(" ")[1]
    
    third = df_group.loc[2].copy()
    third["Group"] = letra
    
    third_teams.append(third)

df_third = pd.DataFrame(third_teams)

# Ordenar terceros (criterio FIFA)
df_third = df_third.sort_values(
    by=["Pts"],
    ascending=False
).reset_index(drop=True)

# Elegir 8 mejores
best_thirds = df_third.head(8)


########################################################
# 2️⃣ CREAR DICCIONARIO DE REEMPLAZO
########################################################

reemplazos = {}

# 1° y 2° de cada grupo
for group in dict_table:
    df_group = dict_table[group]
    letra = group.split(" ")[1]
    
    reemplazos[f"1{letra}"] = df_group.loc[0, "Selección"]
    reemplazos[f"2{letra}"] = df_group.loc[1, "Selección"]

# 8 mejores terceros
for _, row in best_thirds.iterrows():
    reemplazos[f"3{row['Group']}"] = row["Selección"]

########################################################
# 4️⃣ REEMPLAZAR EN FIXTURE
########################################################

for col in ["home", "away"]:
    
    df_fixture_dieciseisavos[col] = (
        df_fixture_dieciseisavos[col]
        .replace(reemplazos)
        .apply(lambda x: resolver_mejor_tercero(x, df_third))
    )

print("✅ 16avos listos")
print(df_fixture_dieciseisavos)


✅ 16avos listos
              home       score           away  year  match_number
72          México  Partido 73         Canadá  2026            73
73        Alemania  Partido 74        Escocia  2026            74
74    Países Bajos  Partido 75      Marruecos  2026            75
75          Brasil  Partido 76          Japón  2026            76
76         Francia  Partido 77        Escocia  2026            77
77         Ecuador  Partido 78        Senegal  2026            78
78       Sudáfrica  Partido 79        Escocia  2026            79
79      Inglaterra  Partido 80        Argelia  2026            80
80        Paraguay  Partido 81        Argelia  2026            81
81         Bélgica  Partido 82  Corea del Sur  2026            82
82        Colombia  Partido 83        Croacia  2026            83
83          España  Partido 84        Austria  2026            84
84           Suiza  Partido 85         Egipto  2026            85
85       Argentina  Partido 86        Uruguay  2026         

In [103]:

df_fixture_dieciseisavos=simular_eliminatoria(df_fixture_dieciseisavos)
df_fixture_dieciseisavos

,home,score,away,year,match_number,Goals_home,Goals_away,Winner
72,México,2.3664240353516286-0.44988079229274724,Canadá,2026,73,2.366424,0.449881,México
73,Alemania,2.5561003838370504-0.33617529498737375,Escocia,2026,74,2.556100,0.336175,Alemania
74,Países Bajos,2.188169874152482-0.6119352440029313,Marruecos,2026,75,2.188170,0.611935,Países Bajos
75,Brasil,2.395844094538403-0.457493046427756,Japón,2026,76,2.395844,0.457493,Brasil
76,Francia,2.4261371781085823-0.4462731375605059,Escocia,2026,77,2.426137,0.446273,Francia
77,Ecuador,1.4372653681864287-1.3182038669903966,Senegal,2026,78,1.437265,1.318204,Ecuador
78,Sudáfrica,1.539122394177043-1.258174205113065,Escocia,2026,79,1.539122,1.258174,Sudáfrica
79,Inglaterra,2.1131425810091433-0.6827961001830497,Argelia,2026,80,2.113143,0.682796,Inglaterra
80,Paraguay,1.5525232249385836-1.2115337695551744,Argelia,2026,81,1.552523,1.211534,Paraguay
81,Bélgica,2.1034023210056203-0.7280321310634322,Corea del Sur,2026,82,2.103402,0.728032,Bélgica


## ARMO LAS LLAVES Y SIMULO OCTAVOS

In [104]:
df_fixture_octavos=asignar_ganador(df_fixture_dieciseisavos,df_fixture_octavos)
df_fixture_octavos


,home,score,away,year,match_number
88,Alemania,Partido 89,Francia,2026,89
89,México,Partido 90,Países Bajos,2026,90
90,Brasil,Partido 91,Ecuador,2026,91
91,Sudáfrica,Partido 92,Inglaterra,2026,92
92,Croacia,Partido 93,España,2026,93
93,Paraguay,Partido 94,Bélgica,2026,94
94,Argentina,Partido 95,Estados Unidos,2026,95
95,Suiza,Partido 96,Portugal,2026,96


In [105]:
df_fixture_octavos=simular_eliminatoria(df_fixture_octavos)
df_fixture_octavos

,home,score,away,year,match_number,Goals_home,Goals_away,Winner
88,Alemania,1.6116959318364272-1.1926583528751795,Francia,2026,89,1.611696,1.192658,Alemania
89,México,0.4096294992294369-2.4567890347308383,Países Bajos,2026,90,0.409629,2.456789,Países Bajos
90,Brasil,2.138026111328035-0.6708870701918799,Ecuador,2026,91,2.138026,0.670887,Brasil
91,Sudáfrica,0.6316213273153295-2.194142937148101,Inglaterra,2026,92,0.631621,2.194143,Inglaterra
92,Croacia,1.3010008144593783-1.4729165005989813,España,2026,93,1.301001,1.472917,España
93,Paraguay,1.2389052346044227-1.543332556104818,Bélgica,2026,94,1.238905,1.543333,Bélgica
94,Argentina,2.3588521872347417-0.5011593620153886,Estados Unidos,2026,95,2.358852,0.501159,Argentina
95,Suiza,0.652578811516058-2.1947100999571103,Portugal,2026,96,0.652579,2.194710,Portugal


## ARMO LAS LLAVES Y SIMULO CUARTOS

In [106]:
df_fixture_cuartos=asignar_ganador(df_fixture_octavos,df_fixture_cuartos)
df_fixture_cuartos

,home,score,away,year,match_number
96,Alemania,Partido 97,Países Bajos,2026,97
97,España,Partido 98,Bélgica,2026,98
98,Brasil,Partido 99,Inglaterra,2026,99
99,Argentina,Partido 100,Portugal,2026,100


In [107]:
df_fixture_cuartos=simular_eliminatoria(df_fixture_cuartos)
df_fixture_cuartos

,home,score,away,year,match_number,Goals_home,Goals_away,Winner
96,Alemania,1.4060427657474142-1.3786406940389362,Países Bajos,2026,97,1.406043,1.378641,Alemania
97,España,1.8581709772714745-0.9452096880711398,Bélgica,2026,98,1.858171,0.945210,España
98,Brasil,1.752910566354098-1.0242954404017728,Inglaterra,2026,99,1.752911,1.024295,Brasil
99,Argentina,1.410112168380177-1.383511282148513,Portugal,2026,100,1.410112,1.383511,Argentina


## ARMO LAS LLAVES Y SIMULO SEMIS 

In [108]:
df_fixture_semis=asignar_ganador(df_fixture_cuartos,df_fixture_semis)
df_fixture_semis

,home,score,away,year,match_number
100,Alemania,Partido 101,España,2026,101
101,Brasil,Partido 102,Argentina,2026,102


In [110]:
df_fixture_semis=simular_eliminatoria(df_fixture_semis)
df_fixture_semis


,home,score,away,year,match_number,Goals_home,Goals_away,Winner
100,Alemania,1.7396513959066444-1.060945060013017,España,2026,101,1.739651,1.060945,Alemania
101,Brasil,1.8278012599070455-0.9778799951819321,Argentina,2026,102,1.827801,0.977880,Brasil


## ARMO LAS LLAVES Y SIMULO LA FINAL

In [116]:
df_fixture_final=asignar_perdedor(df_fixture_semis,df_fixture_final)
df_fixture_final

,home,score,away,year,match_number
102,España,Partido 103,Argentina,2026,103
103,Alemania,Partido 104,Brasil,2026,104


In [117]:
df_fixture_final=simular_eliminatoria(df_fixture_final)
df_fixture_final


,home,score,away,year,match_number,Goals_home,Goals_away,Winner
102,España,1.323089731357901-1.464507117545597,Argentina,2026,103,1.323090,1.464507,Argentina
103,Alemania,1.2338706202000802-1.5640871584921476,Brasil,2026,104,1.233871,1.564087,Brasil


In [124]:
final = df_fixture_final[df_fixture_final["match_number"] == 104]

print(f"🏆 GANADOR DEL MUNDIAL 2026: {final['Winner'].values[0]}")


🏆 GANADOR DEL MUNDIAL 2026: Brasil
